# Referencias
Me baso en dos fuentes distintas, una para el algoritmo completo y otra para el tratamiento de la cola:
- Marsaglia, G., & Tsang, W. W. (2000). The ziggurat method for generating random variables. Journal of Statistical Software, 5(8), 1–7.
- Marsaglia, G. (1963). Generating a variable from the tail of the normal distribution (Mathematical Note No. 322). Boeing Scientific Research Laboratories.

# Procedimiento
## Intención
Se implementará un algoritmo Ziggurat y una versión modificada para tratar mejor la cola de la función de densidad que estemos generando, ambas versiones se compararán con otras dos funciones en la generación de una exponencial y luego usaremos estas versiones de Ziggurat para hacer la tabla de densidad de una distribución normal estándar.

## Descripción de los algoritmos

### Algoritmo de Ziggurat Estándar
#### Precálculo
Partimos de nuestro conjunto $C$ que es el área bajo la curva $y=f(x)=\lambda e^{- \lambda x}$. La idea es generar un área $Z$$ tal que $Z \supset C$, este área la conformamos por un conjunto $N$ de cajas con mismo área $v$.
Para maximizar la eficiencia se busca que $N$ sea un múltiplo de 2, de tal manera que se pueda aprovechar la estructura binaria de los ordenadores, y poder asi tomar valores truncando ciertos valores, pero nos estamos adelantando. La idea es que las cajas sean 32, 64, 128, 256, etc. En este caso usaremos 255 para la exponencial concretamente, para la normal se usa 256.

Para generar las cajas en base a nuestro N debemos seguir una relación concreta: TODAS las cajas deben tener área $v$. Definimos las cajas tal que:
$$x_0 = 0 < x_1 < x_2 < ... < x_N = r$$
$$v = x_i [f(x_{i-1}) - f(x_i)] \qquad i= 1, 2, ..., N$$
Esto se define así para todas las cajas, pero para el caso $x_N = r$ no solo contiene la caja, contiene además el área de la cola de la función decreciente, tal que:
$$v = r f(r) +  \int_{r}^{\infty} f(r), dx $$

##### Cálculo de r
Como se puede ver, aún teniendo v dado, este depende de r, por lo que los investigadores nos han proporcionado la función $z(r)$ que nos permite ver si el valor de $r$ que estamos utilizando cumple con las definicinoes y todas las cajas que genera tiene área v. Esta función se define como:

*Algorithm: $z(r)$ para el cálculo del valor óptimo de $r$*  

*Input:*  
$f(x)$ (densidad decreciente),  
$f^{-1}(y)$ (inversa de $f$),  
$N$ (número de rectángulos).  

*Output:*  
$z(r)$ (valor que debe anularse para obtener el $r$ correcto).  

---

1. $( x_N \leftarrow r )$

2. $( v \leftarrow r,\quad f(r) + \displaystyle\int_{r}^{\infty} f(x),dx )$

3. *for* $i = N-1, N-2, \dots, 1$ 
   
   *do:*  
   $\quad x_i \leftarrow f^{-1}\!\left(f(x_{i+1}) + \dfrac{v}{x_{i+1}}\right ) $

4. *return* $z(r) = v - \left(x_1 - x_1 f(x_1)\right) $


---

##### Valores por defecto y versiones empleadas

En este ejercicio, como vamos a comparar con la **distribución exponencial** y más adelante generaremos la **distribución normal estándar**, **no calcularemos el valor de \( r \)** ni el área $v$** de forma numérica**.  
Para simplificar el trabajo, utilizaremos directamente los valores de referencia proporcionados por Marsaglia y Tsang (2000), que son los que usan la mayoría de implementaciones modernas del método de Ziggurat.

- **Exponencial estándar (λ = 1)**  
  $f(x) = e^{-x}, \quad x \ge 0 $  
  Se trata de la versión clásica de la exponencial decreciente.  
  En este caso se utilizan **255 rectángulos (N = 255)**, con los valores:
  $$
  r = 7.69711747013104972, \qquad
  v = 0.003949659822581557
  $$
  La eficiencia del método con estos parámetros es del **98.9 %**.


- **Normal estándar (mitad positiva)**  
  $ f(x) = e^{-x^2/2}, \quad x \ge 0 $  
  Se usa únicamente la mitad positiva de la normal estándar (sin constante de normalización).  
  Para obtener la distribución completa $ N(0,1) $, basta con asignar un signo aleatorio $ \pm1 $ con probabilidad $ 1/2 $.  
  En este caso se utilizan **256 rectángulos (N = 256)**, con los valores:
  $$
  r = 3.6541528853610088, \qquad
  v = 0.00492867323399
  $$
  La eficiencia del método con estos parámetros es del **99.33 %**.


Estos valores provienen directamente del trabajo original de *Marsaglia y Tsang (2000), “The Ziggurat Method for Generating Random Variables”*, y nos permiten concentrarnos en la comprensión y comparación del método sin añadir complejidad innecesaria en el cálculo de $r $ y $ v $.

##### Cálculo de las cajas
En este punto ya tenemos r, ya conocemos la relación entre una caja y la anterior y conocemos las restricciones (v) de estas cajas, solo tenemos que invertir la funcion para conseguir calcular todas las cajas:
$$f(x_{i-1}) = f(x_i) + \frac{v}{x_i}$$
$$x_{i-1} = f^{-1}(f(x_i) + \frac{v}{x_i})$$

#### Algoritmo Generador

Ahora que ya tenemos las cajas (y suponemos que tienes un generador de la distribuión uniforme funcional) podemos mostrar como es el algoritmo de Ziggurat a la hora de generar distribuciones.

---

*Algorithm: Generación de una variable aleatoria mediante el método de Ziggurat (con tratamiento de colas de Marsaglia, 1963)*  

*Input:*  
$f(x)$ (función de densidad decreciente),  
$\{x_i\}, \{f_i\}$ (tablas precalculadas),  
$r, v, N$ (parámetros del ziggurat).  

*Output:*  
$x$ (valor aleatorio con la distribución objetivo).  

---

1. Generar un entero aleatorio de 32 bits $j$.

2. Obtener el índice de caja:
   $$
   i \leftarrow j \; \&\; (N-1)
   $$
   (usa los bits bajos de $j$).

3. Calcular el candidato:
   $$
   x \leftarrow |j| \cdot w[i]
   $$
   donde $w[i] = \dfrac{x_i}{2^{32}}$.

4. **Comprobación rápida:**  
   Si $|j| < k[i]$ entonces **aceptar directamente** y devolver $x$  
   (se encuentra en la parte segura del rectángulo, bajo la curva).

5. **Si no se cumple:**
   - **Caso cola (i = 0):**
     1. Generar $u_1, u_2 \sim U(0,1)$.
     2. Calcular
        $$
        x \leftarrow r + \frac{-\ln(u_1)}{r}
        $$
     3. Aceptar si
        $$
        u_2 < e^{-\frac{1}{2}(x^2 - r^2)}.
        $$
        Si no, repetir desde el paso 5.

   - **Caso caja interna (i > 0):**
     1. Calcular $f(x) = e^{-x^2/2}$ (o la correspondiente para la distribución).
     2. Aceptar si
        $$
        f(x) > f_i + \dfrac{f_{i-1} - f_i}{x_i - x_{i-1}}(x - x_i)
        $$
        (el punto está bajo la curva).  
        Si no, repetir desde el paso 1.

6. Para la **normal**, asignar signo aleatorio:
   $$
   x \leftarrow \pm x \quad \text{con prob. } 0.5
   $$

7. Devolver $x$.



### Algoritmo de Ziggurat con tratado de colas